In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T

from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.transforms import Compose
from torch_geometric.utils import dense_to_sparse, negative_sampling
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.optim import Adam

/home/ealvarez/miniconda3/envs/graph_matching/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Comparison MS data

In [2]:
dataset1 = "deybis_05" # "deybis_05", "vanessa_05", "mentos_05", "Douban Online_Offline", "ACM_DBLP" # args.dataset
dataset2 = "vanessa_05" # "deybis_05", "vanessa_05", "mentos_05", "Douban Online_Offline", "ACM_DBLP" # args.dataset

encoder = "GIN"


In [3]:
list_train_set = [
	["FrescoAmazonas_1", "FrescoAmazonas_2"],
	["FrescoCusco_1", "FrescoCusco_2"],
	["FrescoSanMartin_1", "FrescoSanMartin_2"],
	["SecoAmazonas_1", "SecoAmazonas_2"],
	["SecoCusco_1", "SecoCusco_2"],
	["SecoSanMartin_1", "SecoSanMartin_2"]
]

In [10]:
tol_rt, tol_mz = 0.5, 0.01 # Change

for train_set in list_train_set:
	df_1 = pd.read_csv(f"data/{dataset1}/output/node_alignment_{encoder}_{dataset1}_{train_set}.csv" , sep=";", decimal=",")
	df_2 = pd.read_csv(f"data/{dataset2}/output/node_alignment_{encoder}_{dataset2}_{train_set}.csv" , sep=";", decimal=",")

	df_match = (
		df_1.assign(key=1)
			.merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
			.drop(columns="key")
	)

	rt_diff = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
	mz_diff = (
		(df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() # / df_match["Average Mz_2"] * 1e6
	)
	# mask = (rt_diff <= tol_rt) & (mz_diff <= tol_mz)
	mask = (mz_diff <= tol_mz)

	df_intersection = df_match.loc[mask].reset_index(drop=True)
	print(train_set)
	display(df_intersection)

['FrescoAmazonas_1', 'FrescoAmazonas_2']


,Id_1,Average Rt_1,Average Mz_1,Metabolite name_1,Id_2,Average Rt_2,Average Mz_2,Metabolite name_2
0,1477,21.549,341.12357,Unknown,342,12.927,341.1137,[Similar to: 2-(Acetylamino)-3-[3-(cyclopentyl...


['FrescoCusco_1', 'FrescoCusco_2']


,Id_1,Average Rt_1,Average Mz_1,Metabolite name_1,Id_2,Average Rt_2,Average Mz_2,Metabolite name_2
0,282,7.369,327.00706,BIS(TRIMETHYLSILYLMETHYL) SULFIDE; EI-B; MS,442,19.382,327.00874,NaN
1,1678,24.495,327.00652,Unknown,442,19.382,327.00874,NaN


['FrescoSanMartin_1', 'FrescoSanMartin_2']


...

,Id_1,Average Rt_1,Average Mz_1,Metabolite name_1,Id_2,Average Rt_2,Average Mz_2,Metabolite name_2
0,374,8.076,327.01132,"1,4-BIS(TRIMETHYLSILYL)BENZENE; EI-B; MS",442,19.382,327.00874,NaN
1,1477,21.549,341.12357,Unknown,342,12.927,341.11370,[Similar to: 2-(Acetylamino)-3-[3-(cyclopentyl...


['SecoCusco_1', 'SecoCusco_2']


,Id_1,Average Rt_1,Average Mz_1,Metabolite name_1,Id_2,Average Rt_2,Average Mz_2,Metabolite name_2
0,185,6.772,281.07687,Unknown,41,2.100,281.07077,sulfalene
1,190,6.807,183.08495,Unknown,285,11.334,183.07844,Triethyl phosphate
2,1600,23.308,341.10735,Unknown,342,12.927,341.11370,[Similar to: 2-(Acetylamino)-3-[3-(cyclopentyl...
3,36,6.154,281.06985,Unknown,41,2.100,281.07077,sulfalene


['SecoSanMartin_1', 'SecoSanMartin_2']


,Id_1,Average Rt_1,Average Mz_1,Metabolite name_1,Id_2,Average Rt_2,Average Mz_2,Metabolite name_2
0,1146,17.480,281.07797,Unknown,41,2.100,281.07077,sulfalene
1,1191,18.013,251.18527,Unknown,217,9.137,251.18581,NaN
2,1477,21.549,341.12357,Unknown,342,12.927,341.11370,[Similar to: 2-(Acetylamino)-3-[3-(cyclopentyl...
3,1538,22.373,281.07199,Unknown,41,2.100,281.07077,sulfalene
4,1760,25.762,341.16177,Unknown,313,11.951,341.15778,"3,8,15-Trihydroxy-12,13-epoxytrichothec-9-en-4..."


In [8]:
tol_rt, tol_mz = 0.5, 0.1 # Change

df_1 = pd.read_csv(f"data/{dataset1}/raw.csv", index_col=0).iloc[:, :3]
df_2 = pd.read_csv(f"data/{dataset2}/raw.csv", index_col=0).iloc[:, :3]

df_match = (
    df_1.assign(key=1)
        .merge(df_2.assign(key=1), on="key", suffixes=("_1", "_2"))
        .drop(columns="key")
)

rt_diff = (df_match["Average Rt_1"] - df_match["Average Rt_2"]).abs()
mz_diff = (
    (df_match["Average Mz_1"] - df_match["Average Mz_2"]).abs() # / df_match["Average Mz_2"] * 1e6
)
# mask = (rt_diff <= tol_rt) & (mz_diff <= tol_mz)
mask = (mz_diff <= tol_mz)

df_intersection = df_match.loc[mask].reset_index(drop=True)
display(df_intersection)

,Average Rt_1,Average Mz_1,Metabolite name_1,Average Rt_2,Average Mz_2,Metabolite name_2
0,6.068,159.99556,Unknown,3.088,160.07606,1H-indene-3-carboxamide
1,6.068,159.99556,Unknown,3.114,160.04529,NaN
2,6.068,159.99556,Unknown,3.242,160.04522,NaN
3,6.098,234.15141,Unknown,10.904,234.20692,"[Similar to: Bis(1-octyloxy-2,2,6,6-tetramethy..."
4,6.103,281.04929,Unknown,2.100,281.07077,sulfalene
...,...,...,...,...,...,...
660,30.806,281.08865,Unknown,2.112,280.99604,NaN
661,30.824,327.02597,Unknown,19.382,327.00874,NaN
662,30.864,341.05097,Unknown,12.927,341.11370,[Similar to: 2-(Acetylamino)-3-[3-(cyclopentyl...
663,30.875,326.96420,Unknown,19.382,327.00874,NaN
